In [1]:
import os
import pandas as pd

import logging
import logging.config
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder

import ast

PATH_AGROCITTCA_KEY = r'G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON\_keys\agrocittca.data'
PATH_DB_KEY = r'G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON\_keys\llave_db_agrocittca'
PATH_OUTPUT = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AGRO-CITTCA\_DATA_ANALISIS_AGROCITTCA'

In [2]:
# cargar datos para conectar a AGROCITTCA
data = open(PATH_AGROCITTCA_KEY,'r')
password = data.readlines()[0]
data.close()

# extraccion de los datos
datos_cnx = ast.literal_eval(password)

# lectura de llave ssh
mypkey = paramiko.RSAKey.from_private_key_file(PATH_DB_KEY, password='micontrasena')
# ssh variables
ssh_host = datos_cnx['ssh_host']
ssh_port = datos_cnx['ssh_port']
localhost = datos_cnx['localhost']
localport = datos_cnx['localport']
ssh_user = datos_cnx['ssh_user']
# database variables
sql_username = datos_cnx['sql_username']
sql_password = datos_cnx['sql_password']
sql_main_database = datos_cnx['sql_main_database']

In [5]:
# funcon para ejecutar una script SQL
def dbQuery(q):
    """ access the database over the SSH tunnel and execute the query """
    logger = logging.getLogger()
    logger.debug("Connecting to reporting server...", end="")
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(localhost, 3306)) as tunnel:
        logger.debug("Connected")
        logger.debug("Connecting to database...")
        try:
            conn = pymysql.connect(
                host='127.0.0.1', 
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor)
            logger.debug("Connected")
            logger.debug("Executing query...")
            cursor = conn.cursor()
            cursor.execute(q)
            data = cursor.fetchall()
            logger.debug("Done")
        finally:
            conn.close()
    return data

def get_activos():
    query_1 = '''select 
                    DATE(tok.created_at) as created_at, 
                    usu.codigo, 
                    usu.name, 
                    usu.apellido_paterno, 
                    usu.apellido_materno, 
                    usu.username, 
                    usu.tipo, 
                    \'reg acces\' as origen
                from oauth_access_tokens tok
                inner join users usu on tok.user_id=usu.id
                WHERE 
                    tok.created_at >= DATE_SUB(NOW(), INTERVAL 6 MONTH) and 
                    usu.tipo=\'propietario\''''
    usuarios_acces = dbQuery(query_1)
    data_acces = pd.DataFrame(usuarios_acces)
    
    query_2 = '''select 
                    DATE(ac.created_at) as created_at, 
                    us.codigo, 
                    us.name, 
                    us.apellido_paterno, 
                    us.apellido_materno, 
                    us.username, us.tipo, 
                    \'reg. labor\'  as origen\
                from actividades ac
                inner join users us on us.id = ac.usuario_registro
                WHERE 
                    ac.created_at >= DATE_SUB(NOW(), INTERVAL 6 MONTH)'''
    usuarios_reg = dbQuery(query_2)
    data_reg = pd.DataFrame(usuarios_reg)
    data = pd.concat([data_acces, data_reg], ignore_index=True)
    # split de name [cod - nombre]
    data[['cod', 'nombre']] = data['name'].str.split(' - ', n=1, expand=True)
    # concatener con ap materno
    data['canero'] = data['nombre'] + data['apellido_materno']
    # ordenar las filas
    data = data[['created_at', 'cod', 'canero', 'username', 'tipo', 'origen']]
    data['cod'] = data['cod'].astype(int)
    data.to_excel(PATH_OUTPUT + "/usuarios_activos.xlsx", index=False)
    #return data

In [6]:
get_activos()